### Práctica de Bosques Japoneses

### Partiendo del conjunto de datos indicado y el código inicial que se ofrece para la carga de datos, realice las siguientes tareas:

#### Fuente https://archive.ics.uci.edu/dataset/333/forest+type+mapping 

#### 1.- Realizar análisis de PCA para detectar los componentes más relevantes. Visualizar los CVRs y decidir los PCs a usar (por ejemplo 3) 
#### 2.- Con los datos obtenidos, realizar análisis de clustering, en el que se emplee varios algoritmos vistos: K.Means, Clustering Jerárquico (Aglomerativo), DFSCAN (Anadir librerías necesarias)
#### 3.- Realizar diferentes pruebas con hiperparámetros de dichos algoritmos de clustering para obtener agrupaciones optimizadas
#### 3.- Generar visualizaciones (por ejemplo, scatter plots, heatmaps, etc.) sobre los resultados de los algoritmos de PCA y clustering
#### 4.- Incluir breves razonamientos explicativos de los resultados

In [1]:
import numpy as np 
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
from sklearn.cluster import KMeans
from sklearn.cluster import AgglomerativeClustering
from sklearn.cluster import DBSCAN
from scipy.cluster.hierarchy import linkage, dendrogram, fcluster
warnings.filterwarnings(action='ignore') 
%matplotlib inline

In [2]:
# Load data
df = pd.read_csv('bosques-japoneses.csv', header='infer')

#### Este conjuto de datos contiene 198 muestras con 27 variables y 4 clases de clasificación,

In [6]:
df_features =  df.drop(columns=['class'])
df_target = df['class']

In [8]:
df_features

,b1,b2,b3,b4,b5,b6,b7,b8,b9,pred_minus_obs_H_b1,...,pred_minus_obs_H_b9,pred_minus_obs_S_b1,pred_minus_obs_S_b2,pred_minus_obs_S_b3,pred_minus_obs_S_b4,pred_minus_obs_S_b5,pred_minus_obs_S_b6,pred_minus_obs_S_b7,pred_minus_obs_S_b8,pred_minus_obs_S_b9
0,39,36,57,91,59,101,93,27,60,75.70,...,-2.36,-18.41,-1.88,-6.43,-21.03,-1.60,-6.18,-22.50,-5.20,-7.86
1,84,30,57,112,51,98,92,26,62,30.58,...,-2.26,-16.27,-1.95,-6.25,-18.79,-1.99,-6.18,-23.41,-8.87,-10.83
2,53,25,49,99,51,93,84,26,58,63.20,...,-1.46,-15.92,-1.79,-4.64,-17.73,-0.48,-4.69,-19.97,-4.10,-7.07
3,59,26,49,103,47,92,82,25,56,55.54,...,2.68,-13.77,-2.53,-6.34,-22.03,-2.34,-6.60,-27.10,-7.99,-10.81
4,57,49,66,103,64,106,114,28,59,59.44,...,-2.94,-21.74,-1.64,-4.62,-23.74,-0.85,-5.50,-22.83,-2.74,-5.84
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
193,55,61,90,75,55,91,84,24,54,57.14,...,1.28,-20.84,-1.47,-5.72,-18.73,-1.18,-5.22,-17.81,-1.28,-3.87
194,66,69,93,90,63,100,99,25,57,45.97,...,-1.50,-19.64,-1.45,-5.60,-17.03,-1.08,-5.15,-16.77,-1.46,-4.03
195,79,30,55,115,51,96,109,25,59,35.40,...,-3.66,-22.40,-3.91,-6.61,-36.29,-2.13,-5.77,-23.32,-2.09,-4.13
196,69,27,53,106,47,90,87,24,55,35.81,...,-0.88,-13.32,0.16,-2.39,-7.12,0.02,-2.06,-10.04,-0.74,-2.88


In [10]:
df_target

0      d 
1      h 
2      s 
3      s 
4      d 
       ..
193    d 
194    d 
195    h 
196    h 
197    h 
Name: class, Length: 198, dtype: object

In [14]:
standardized = StandardScaler()
standardized.fit(df_features)
scaled_data = standardized.transform(df_features)
X = scaled_data

In [16]:
X

array([[-1.8787969 , -0.28223274, -0.3865936 , ..., -0.6987643 ,
        -2.01261582, -1.87322805],
       [ 1.6513761 , -0.61954914, -0.3865936 , ..., -0.87250077,
        -4.04786229, -3.37519212],
       [-0.78052085, -0.90064614, -0.84980409, ..., -0.2157387 ,
        -1.40259644, -1.47371572],
       ...,
       [ 1.25913466, -0.61954914, -0.50239623, ..., -0.85531804,
        -0.28792467,  0.01307699],
       [ 0.47465177, -0.78820734, -0.61819885, ...,  1.68008902,
         0.46073548,  0.64521675],
       [ 1.33758295, -0.67576854, -0.50239623, ..., -0.39520276,
         0.37200539,  0.23053307]])